## Домашнее задание по Автобрее №1
### Настя Чижикова, БКЛ181

СИТУАЦИЯ: я написала код, который ывкачивает негативные и позитивные отзывы с сайта IRecommend, но в какой-то момент он стал меня банить за много запросов (паузы не помогали). Я переписала код совсем немного, чтобы он работал на сайте Отзовик.ру, и он работал, но потом тоже меня забанил и не хотел ничего выдавать. Поэтому в итоге я сделала корпус вручную (в приложенных файлах). Код для парсинга оставляю на память.

#### Возьмем набор отзывов о самых мультфильмах на Отзовике

In [40]:
from bs4 import BeautifulSoup
from html import unescape
import sqlite3
import collections
import re
import requests
from pprint import pprint
import time
import nltk
from pymorphy2 import MorphAnalyzer
import random

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

In [41]:
headers = {'User-Agent': ua.random}
result = requests.get('https://otzovik.com/show_filter.php?cat_id=182&f[r]=10_')
result.encoding = 'utf-8'
html = result.text

In [42]:
soup = BeautifulSoup(html,'html.parser')

In [43]:
links = []
for ps in soup.find_all('a', "product-name"):
    links.append(ps.get('href'))

In [44]:
links

[]

In [32]:
def open_all_reviews(link):  #  функция возвращает ссылки на полные версии отзывов
    result = requests.get(link)
    result.encoding = 'utf-8'
    html = result.text
    soup = BeautifulSoup(html,'html.parser')
    time.sleep(20)
    
    
    full_reviews = []
    for ps in soup.find_all('a', "review-read-link"):
        full_reviews.append(ps.get('href'))
        
    print(full_reviews)
    return full_reviews

In [33]:
def get_reviews(link):  #  функция собирает тексты отзывов
    neg_revs = []
    pos_revs = []
    
    full_reviews = open_all_reviews(link)
    
    
    for review in full_reviews:
        time.sleep(20)
        rev_link = "https://otzovik.com" + review
        result = requests.get(rev_link)
        print(rev_link)
        result.encoding = 'utf-8'
        html = result.text
        soup = BeautifulSoup(html,'html.parser')
        
        
        m = soup.find('meta', itemprop="ratingValue")
        rating = int(m.get('content'))
        
        rev_text = soup.find('div', '46pr1').contents
            
        if rating > 3:  #  за позитивные отзывы считаем только выше 3
            pos_revs.append(rev_text)
        else:
            neg_revs.append(rev_text)
                
        
                
    return pos_revs, neg_revs

In [34]:
neg_reviews = []
pos_reviews = []
valid_reviews = [[], []]  #  выборка для проверки качества

for l in links:
    if len(neg_reviews) < 30 or len(pos_reviews) < 30 or (len(valid_reviews[0]) + len(valid_reviews[1])) < 10:
        link = "https://otzovik.com" + l
        pos, neg = get_reviews(link)
    
        if len(neg_reviews) < 30:
            neg_reviews.extend(neg)
        if len(pos_reviews) < 30:
            pos_reviews.extend(pos)
        
        if len(neg_reviews) >= 30 and len(pos_reviews) >= 30:
            if len(valid_reviews[0]) < 5:
                valid_reviews[0].extend(neg)
            if len(valid_reviews[1]) < 5:
                valid_reviews[1].extend(pos)
    
    

[]
[]
[]


KeyboardInterrupt: 

Корпус из ~30 негативных и ~30 позитивных отзывов на разные современные мультфильмы собран вручную с сайта Отзовик.ру, за позитивные отзывы считались отзывы с оценкой 4 или 5 звезд, за негативные - 3 и меньше

In [18]:
with open('neg_reviews.txt', encoding='utf-8') as f:
    neg_reviews = f.read()

In [19]:
with open('pos_reviews.txt', encoding='utf-8') as f:
    pos_reviews = f.read()

### Обработка

In [22]:
m = MorphAnalyzer()

In [26]:
def preprocess(text):
    words = []
    for w in nltk.word_tokenize(text.lower()):
        words.append(m.parse(w)[0].normal_form)
    return words

In [45]:
neg_words_ = preprocess(neg_reviews)
pos_words_ = preprocess(pos_reviews)

In [47]:
neg_words = collections.Counter()
pos_words = collections.Counter()

for w in neg_words_:
    neg_words[w] += 1
    
for w in pos_words_:
    pos_words[w] += 1

In [63]:
uni_pos = collections.Counter()
uni_neg = collections.Counter()

for w in pos_words:  #  сохраним уникальные элементы с их частотностью
    if w not in neg_words:
        uni_pos[w] = pos_words[w]
        
for w in neg_words: 
    if w not in pos_words:
        uni_neg[w] = neg_words[w]

In [64]:
def predict(rev):
    rev_words = preprocess(rev)
    vect = []  #  создаем вектор для отзыва
    
    for w in rev_words:
        if w in uni_pos:
            vect.append(1)
        elif w in uni_neg:
            vect.append(-1)
        else:
            vect.append(0)
    
    pred = sum(vect)
    
    
    '''Если слов из "положительного" списка больше, сумма будет положительная, и наоборот.
    Если слов поровну (или все слова встречаются впервые), возвращаем ноль'''
    
    
    if pred > 0:
        return 1
    elif pred < 0:
        return -1
    else:
        return 0

#### Считаем accuracy на (тоже вручную собранной) валидационной выборке

In [65]:
valid_reviews = [[], []]
with open('val_neg_reviews.txt', encoding='utf-8') as f:
    neg = f.read()
    valid_reviews[0] = neg.split('\n\n')
    
with open('val_pos_reviews.txt', encoding='utf-8') as f:
    pos = f.read()
    valid_reviews[1] = pos.split('\n\n')

In [66]:
def accuracy():
    acc_v = []

    for r in valid_reviews[0]:
        if predict(r) == -1:
            acc_v.append(1)
        elif predict(r) == 1:
            acc_v.append(0)
        
    for r in valid_reviews[1]:
        if predict(r) == -1:
            acc_v.append(0)
        elif predict(r) == 1:
            acc_v.append(1)

    return sum(acc_v) / len(acc_v)

In [67]:
accuracy()

0.5555555555555556

Попробуем то же самое, но сохраним только уникальные слова, встретившиеся больше одного раза (думаю, получится плохо)

In [68]:
uni_pos_ = uni_pos
uni_neg_ = uni_neg

uni_pos = collections.Counter()
uni_neg = collections.Counter()

for w in uni_pos_:
    if uni_pos_[w] > 1:
        uni_pos[w] = uni_pos_[w]
        
for w in uni_neg_:
    if uni_neg_[w] > 1:
        uni_neg[w] = uni_neg_[w]

In [69]:
accuracy()

0.5

Получилось плохо :(

#### Выводы

##### Что можно было сделать

0. Собрать отзывы ~~как нормальный человек~~ не руками и в большем количестве (явно не хватает выборки, частотность слов не сбалансирована и иногда обусловлена сюжетом мультфильма, о котором отзыв 
1. Токенизировать не на слова, а как-то по-другому: в частности, очень хотелось бы сохранять, например, последовательность токенов "не рекомендую" или "не понравилось" (на данном этапе они разъединялись и "сливались" с "рекомендую" и "понравилось")
2. Выбрать другую модель для предсказания, например, обучить логистическую регрессию, чтобы предсказание было чуть более осмысленным